In [ ]:
# ===========================
# etl_analysis.ipynb (code)
# ===========================
# Purpose: Read Telco churn table from Supabase,
#          compute metrics, generate basic visuals,
#          and save analysis_summary.csv

import os
import pandas as pd
import matplotlib.pyplot as plt
from supabase import create_client, Client
from dotenv import load_dotenv

# ------------------------------------------------------
# Supabase client
# ------------------------------------------------------
def get_supabase_client() -> Client:
    """Initialize and return Supabase client."""
    load_dotenv()
    url = os.getenv("SUPABASE_URL")
    key = os.getenv("SUPABASE_KEY")

    if not url or not key:
        raise ValueError("❌ Missing SUPABASE_URL or SUPABASE_KEY in .env")

    return create_client(url, key)


def fetch_telco_data(table_name: str = "telco_churn") -> pd.DataFrame:
    """Fetch full telco_churn table from Supabase into a DataFrame."""
    supabase = get_supabase_client()

    # Large limit to get all rows (7043)
    response = supabase.table(table_name).select("*").limit(10000).execute()

    if not response.data:
        raise ValueError("❌ No data returned from Supabase table")

    df = pd.DataFrame(response.data)
    print(f"✅ Fetched {len(df)} rows from Supabase table '{table_name}'")
    return df


# ------------------------------------------------------
# Metrics & analysis
# ------------------------------------------------------
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute:
    - Overall churn percentage
    - Average MonthlyCharges per Contract
    - Count of customers by tenure_group
    - InternetService distribution
    - Pivot: Churn vs tenure_group

    Returns a long-format DataFrame with columns:
    metric, category, subcategory, value
    """
    rows = []

    # 1) Churn percentage
    if "Churn" in df.columns:
        churn_rate = (df["Churn"] == "Yes").mean() * 100
        rows.append({
            "metric": "churn_percentage_overall",
            "category": "All",
            "subcategory": "",
            "value": round(churn_rate, 2)
        })

    # 2) Average monthly charges per contract
    if "Contract" in df.columns and "MonthlyCharges" in df.columns:
        avg_mc = df.groupby("Contract")["MonthlyCharges"].mean()
        for contract, val in avg_mc.items():
            rows.append({
                "metric": "avg_monthly_charges_by_contract",
                "category": contract,
                "subcategory": "",
                "value": round(val, 2)
            })

    # 3) Count of New / Regular / Loyal / Champion customers
    if "tenure_group" in df.columns:
        tg_counts = df["tenure_group"].value_counts()
        for grp, cnt in tg_counts.items():
            rows.append({
                "metric": "count_customers_by_tenure_group",
                "category": str(grp),
                "subcategory": "",
                "value": int(cnt)
            })

    # 4) Internet service distribution
    if "InternetService" in df.columns:
        is_counts = df["InternetService"].value_counts()
        for svc, cnt in is_counts.items():
            rows.append({
                "metric": "internet_service_distribution",
                "category": svc,
                "subcategory": "",
                "value": int(cnt)
            })

    # 5) Pivot table: Churn vs Tenure Group (counts)
    if "tenure_group" in df.columns and "Churn" in df.columns:
        pivot = pd.pivot_table(
            df,
            index="tenure_group",
            columns="Churn",
            values="tenure",   # any column; we use size
            aggfunc="count",
            fill_value=0
        )
        for tg in pivot.index:
            for churn_val in pivot.columns:
                rows.append({
                    "metric": "churn_count_by_tenure_group",
                    "category": str(tg),
                    "subcategory": str(churn_val),
                    "value": int(pivot.loc[tg, churn_val])
                })

    summary_df = pd.DataFrame(rows)
    return summary_df


# ------------------------------------------------------
# Optional visualizations
# ------------------------------------------------------
def generate_plots(df: pd.DataFrame, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    # 1) Churn rate by Monthly Charge Segment (bar)
    if "monthly_charge_segment" in df.columns and "Churn" in df.columns:
        churn_by_segment = (
            df.groupby("monthly_charge_segment")["Churn"]
              .apply(lambda x: (x == "Yes").mean() * 100)
              .sort_index()
        )

        plt.figure()
        churn_by_segment.plot(kind="bar")
        plt.ylabel("Churn Rate (%)")
        plt.title("Churn Rate by Monthly Charge Segment")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "churn_rate_by_monthly_segment.png"))
        plt.close()
        print("📊 Saved: churn_rate_by_monthly_segment.png")

    # 2) Histogram of TotalCharges
    if "TotalCharges" in df.columns:
        plt.figure()
        df["TotalCharges"].astype(float).hist(bins=30)
        plt.xlabel("TotalCharges")
        plt.ylabel("Frequency")
        plt.title("Histogram of TotalCharges")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "hist_total_charges.png"))
        plt.close()
        print("📊 Saved: hist_total_charges.png")

    # 3) Bar plot of Contract types
    if "Contract" in df.columns:
        plt.figure()
        df["Contract"].value_counts().plot(kind="bar")
        plt.ylabel("Count")
        plt.title("Contract Type Distribution")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "contract_type_distribution.png"))
        plt.close()
        print("📊 Saved: contract_type_distribution.png")

def main():
    df = fetch_telco_data("telco_churn")

    summary_df = compute_metrics(df)

    base_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__))) \
        if "__file__" in globals() else os.getcwd()
    processed_dir = os.path.join(base_dir, "data", "processed")
    os.makedirs(processed_dir, exist_ok=True)

    analysis_csv_path = os.path.join(processed_dir, "analysis_summary.csv")
    summary_df.to_csv(analysis_csv_path, index=False)
    print(f"✅ Analysis summary saved to: {analysis_csv_path}")

    plots_dir = os.path.join(processed_dir, "plots")
    generate_plots(df, plots_dir)

if __name__ == "__main__":
    main()


✅ Fetched 1000 rows from Supabase table 'telco_churn'
✅ Analysis summary saved to: d:\ETL_PIPELINE_TELCO\data\processed\analysis_summary.csv
📊 Saved: churn_rate_by_monthly_segment.png
📊 Saved: hist_total_charges.png
📊 Saved: contract_type_distribution.png
